In [29]:

#import sklearn
from sklearn.cluster import KMeans
import pandas as pd 
import random
import itertools as it
import numpy as np
import timeit 
import time
from sklearn.metrics import adjusted_rand_score
from scipy.optimize import linear_sum_assignment
from google.colab import files
from statistics import mode
import sys

In [2]:
if 'google.colab' in sys.modules:
    uploaded = files.upload()

Saving fashion-mnist_train.csv to fashion-mnist_train.csv


In [3]:
train_data = pd.read_csv('fashion-mnist_train.csv')
X_train = train_data.drop('label', axis=1)
y_train = train_data['label']


In [4]:
def performance_measure3(cluster1,cluster2):
  data = {'cluster1': cluster1,'cluster2': cluster2} 
  clusters = pd.DataFrame(data, index=range(len(cluster1)))
  m = -1*np.array(clusters.groupby(['cluster1','cluster2']).size().unstack(fill_value=0))
  indx, per = linear_sum_assignment(m)
  cost_cluster = -m[indx,per].sum()/len(clusters) 
  return ([cost_cluster,per])

In [ ]:
T= [1000,2000] #no. of samples
k=10
N_pct = [5,10] #sample size 
df = pd.DataFrame(columns=['no_of_sims','sample_size','accuracy_rate','running_time'])
for n in N_pct: 
  N = int((n/100)*len(X_train))
  for B in T: #different number of sample 
    bootstraps_data = np.zeros(shape=(len(X_train),B))
    bootstraps_data[:] = np.nan
    for sim in range(B): 
      start_time = time.time()
      rand_indx = np.random.choice(range(len(X_train)), size=N,replace=False)
      X_subset = X_train.iloc[rand_indx,:]
      y_subset = np.array(y_train.iloc[rand_indx])
      kmean_cluster = KMeans(n_clusters=k).fit(X_subset).labels_
      cost,per= performance_measure3(y_subset, kmean_cluster)
      c = [i for i in range(k)]
      dic = dict(zip(per,c))
      adj_cluster = pd.Series(kmean_cluster).transform(lambda x: dic[x] if x in dic else None)
      bootstraps_data[rand_indx,sim] = adj_cluster
    bootstraps_data = pd.DataFrame(bootstraps_data)
    bootstraps_cluster = bootstraps_data.mode(axis=1,dropna=True)[0]
    running_time= time.time() - start_time
    bootstraps_accuracy_rate = performance_measure3(y_subset,bootstraps_cluster)[0]
    df = df.append({'no_of_sims': B, 'sample_size':N,'accuracy_rate':bootstraps_accuracy_rate,'running_time': running_time},ignore_index=True)
    print (df, running_time)
df

   no_of_sims  sample_size  accuracy_rate  running_time
0      1000.0       3000.0       0.123333     37.573466 37.5734658241272
   no_of_sims  sample_size  accuracy_rate  running_time
0      1000.0       3000.0       0.123333     37.573466
1      2000.0       3000.0       0.126333     37.561200 37.56119966506958
   no_of_sims  sample_size  accuracy_rate  running_time
0      1000.0       3000.0       0.123333     37.573466
1      2000.0       3000.0       0.126333     37.561200
2      1000.0       6000.0       0.120000     38.485426 38.48542618751526


In [6]:
start_time = time.time()
kmeans = KMeans(n_clusters=10).fit(X_train)
kmeans_acc = performance_measure3(y_train,kmeans.labels_)[0]
running_time= time.time() - start_time
kmeans_acc,running_time

(0.5405666666666666, 80.15699481964111)